In [ ]:
%pip install matplotlib
%pip install seaborn
%pip install joblib
%pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Import for Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler # Needed for Logistic Regression
from sklearn.pipeline import Pipeline

# Import for Evaluation and Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, RocCurveDisplay
)

from sklearn.base import clone

# Import for Validation
from sklearn.model_selection import TimeSeriesSplit, cross_validate, train_test_split
from sklearn.inspection import permutation_importance

In [2]:
# ---------------------------------------------------------
# CONFIGURATION: PATHS AND HYPERPARAMETERS
# ---------------------------------------------------------
BRONZE_DIR = "/Users/brockolson/Desktop/STAT766/Stat 766 Final Project"
INPUT_FILE = os.path.join(BRONZE_DIR, "nba_modeling_data_silver.csv")
MODEL_OUTPUT = os.path.join(BRONZE_DIR, "nba_playoff_forest.joblib")
VIS_DIR = os.path.join(BRONZE_DIR, "visualizations")
os.makedirs(VIS_DIR, exist_ok=True)

# ---------------------------------------------------------
# LEAKAGE DEFINITION (UPDATED)
# ---------------------------------------------------------
    # Change this list name and its contents
REQUIRED_RAW_FEATURES = [
    'EFG_PCT', 'FTA_RATE', 'TM_TOV_PCT_x', 'OREB_PCT', 
    'OPP_EFG_PCT', 'OPP_FTA_RATE', 'OPP_TOV_PCT', 
    'OPP_OREB_PCT', 'AST_TO', 'AST_RATIO', 
    'E_PACE', 'PACE_PER40', 'POSS'
] 

def load_and_sanitize(filepath):
    """
    Loads the silver dataset, selects the 13 required RAW features, 
    and handles NaN alignment by filtering the full dataset once.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Input file not found at {filepath}")
    
    print(f"Loading data from {filepath}...")
    df = pd.read_csv(filepath)
    
    # 1. Define all columns needed: Features + Target + Metadata
    all_cols = REQUIRED_RAW_FEATURES + ["MADE_PLAYOFFS"]
    meta_cols = [c for c in ["SEASON", "TEAM_ID", "TEAM_NAME"] if c in df.columns]

    # 2. Create the working DataFrame with only necessary columns
    working_df = df[all_cols + meta_cols].copy()
    
    # 3. Handle NaN Alignment: Drop rows where any REQUIRED_RAW_FEATURE is NaN
    rows_before_drop = working_df.shape[0]
    
    # 🛑 CRITICAL FIX: Filtering the single DataFrame ensures X and Y stay perfectly aligned.
    working_df.dropna(subset=REQUIRED_RAW_FEATURES, inplace=True)
    
    rows_after_drop = working_df.shape[0]
    if rows_before_drop != rows_after_drop:
        print(f"WARNING: Dropped {rows_before_drop - rows_after_drop} rows with NaN values during feature cleanup.")
    
    # 4. Final selection and cleaning
    X_clean = working_df[REQUIRED_RAW_FEATURES].reset_index(drop=True)
    
    # 🛑 Target selection and cleaning (ensuring integer type)
    y_clean = working_df["MADE_PLAYOFFS"].reset_index(drop=True).astype(int)
    
    # Metadata
    meta_clean = working_df[meta_cols].reset_index(drop=True)
    
    print(f"Features Selected ({len(X_clean.columns)}): {list(X_clean.columns)}")
    print(f"Final Data Shapes: X={X_clean.shape}, y={y_clean.shape}")

    # 5. Final check
    if len(X_clean.columns) != 13:
        raise ValueError(f"Feature selection failed. Expected 13 columns, found {len(X_clean.columns)}.")
    
    # The return order ensures X, y, meta matches the main block
    return X_clean, y_clean, meta_clean

def train_forest(X, y):
    """
    Trains a Random Forest Classifier with stratified splitting.
    """
    # Stratify ensures the train/test split has the same % of playoff teams
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Initialize Model
    clf = RandomForestClassifier(
        n_estimators=200,      # More trees for stability
        max_depth=12,          # Prevent extreme overfitting
        min_samples_split=5,   # Conservative splitting
        random_state=42,
        n_jobs=-1
    )
    
    print("Training Random Forest...")
    clf.fit(X_train, y_train)
    return clf, X_train, X_test, y_train, y_test

def run_tscv_metrics(model, X, y, n_splits=5):
    """
    Performs Time Series Cross-Validation (TSCV) and collects 
    all 5 required metrics (Accuracy, Precision, Recall, F1, ROC-AUC) 
    for a single model.
    
    NOTE: X and y are expected to be pure NumPy arrays from the main block
          to ensure clean slicing, which resolves the multilabel ValueError.
    """
    # Lists to store metrics for each fold
    # 🛑 FIX: Define the results dictionary UNCONDITIONALLY at the start
    results = {
        'Accuracy': [], 'Precision': [], 'Recall': [], 'F1 Score': [], 'ROC-AUC': []
    }
    
    # CRITICAL FIX: Implement Scaling for Logistic Regression
    if isinstance(model, LogisticRegression):
        # LogReg needs scaled features (X is assumed to be a NumPy array)
        scaler = StandardScaler()
        X_input = scaler.fit_transform(X)
    else:
        # Tree-based models (RF, GB) do not need scaling
        X_input = X # X_input is also a NumPy array here
        
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    print(f"-> Evaluating {model.__class__.__name__} with TSCV...")
    
    # Iterate through each train/test split generated by TSCV
    for fold, (train_index, test_index) in enumerate(tscv.split(X_input)):
        X_train, X_test = X_input[train_index], X_input[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_train = y_train.ravel()
        y_test = y_test.ravel()
        
        # Clone the model (important to reset weights for each fold)
        clf = clone(model)
        clf.fit(X_train, y_train)
        
        y_pred = clf.predict(X_test)
        # Get probabilities for ROC-AUC (using np.array() for robustness)
        y_probs = np.array(clf.predict_proba(X_test))[:, 1]
        
        # Calculate and store metrics 
        results['Accuracy'].append(accuracy_score(y_test, y_pred))
        
        # 🛑 FIX: Explicitly set average='binary' and pos_label=1 to prevent ValueError
        results['Precision'].append(precision_score(y_test, y_pred, average='binary', pos_label=1))
        results['Recall'].append(recall_score(y_test, y_pred, average='binary', pos_label=1))
        results['F1 Score'].append(f1_score(y_test, y_pred, average='binary', pos_label=1))
        
        results['ROC-AUC'].append(roc_auc_score(y_test, y_probs))

    # Calculate mean and std for final table
    final_results = {}
    for metric, scores in results.items():
        mean_score = np.mean(scores)
        std_dev = np.std(scores)
        # Format the string exactly as requested
        final_results[metric] = f"{mean_score:.4f} (±{std_dev:.4f})"
        
    return final_results

def print_tscv_table(results_dict):
    """Prints the final comparison table in a markdown format."""
    
    model_names = list(results_dict.keys())
    metrics = list(results_dict[model_names[0]].keys())
    
    print("\n" + "="*80)
    print("FINAL TIME SERIES CROSS-VALIDATION RESULTS (Mean Score ± Std Dev)")
    print("="*80)
    
    # Print Header
    header = "| Model | " + " | ".join([f"Avg. {m}" for m in metrics]) + " |"
    print(header)
    print("|" + "---|" * (len(metrics) + 1))
    
    # Print Rows
    for model_name in model_names:
        row = f"| {model_name} | "
        metric_values = [results_dict[model_name][m] for m in metrics]
        row += " | ".join(metric_values) + " |"
        print(row)

def evaluate_performance(clf, X_test, y_test):
    """
    Generates comprehensive metrics and confusion matrix, including ROC-AUC.
    """
    y_pred = clf.predict(X_test)
    y_probs = clf.predict_proba(X_test)[:, 1] # Get probabilities for the positive class (1)
    
    # Calculate ROC-AUC
    roc_auc = roc_auc_score(y_test, y_probs) # <-- NEW CALCULATION
    
    print("\n" + "="*40)
    print("MODEL EVALUATION METRICS")
    print("="*40)
    print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred, average='binary', pos_label=1):.4f} (Low False Positives)")
    print(f"Recall:    {recall_score(y_test, y_pred, average='binary', pos_label=1):.4f} (Low False Negatives)")
    print(f"F1 Score:  {f1_score(y_test, y_pred, average='binary', pos_label=1):.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f} (Area Under the Curve)") # <-- NEW OUTPUT
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion Matrix Visualization
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Miss", "Make"])
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix: Playoff Prediction")
    plt.savefig(os.path.join(VIS_DIR, "confusion_matrix.png"))
    plt.close()


def analyze_random_forest_comparison_plot(X_train, y_train, BRONZE_DIR):
    """
    Trains a final Random Forest model and generates a side-by-side plot
    of Mean Decrease in Impurity (MDI) and Permutation Importance (AUC Drop).
    """
    # Define the Random Forest Model (using parameters from your previous runs)
    rf_model = RandomForestClassifier(
        n_estimators=200, max_depth=12, min_samples_split=5, 
        random_state=42, n_jobs=-1
    )
    
    print("Training final Random Forest model for feature comparison plot...")
    rf_model.fit(X_train, y_train)

    # 1. Calculate Permutation Importance (AUC Drop)
    print("Calculating Permutation Importance (AUC Drop)...")
    perm_result = permutation_importance(
        rf_model, X_train, y_train, 
        n_repeats=10, random_state=42, n_jobs=-1, scoring='roc_auc'
    )
    perm_importances = perm_result.importances_mean
    
    # 2. Get Mean Decrease in Impurity (MDI)
    mdi_importances = rf_model.feature_importances_
    feature_names = X_train.columns
    
    # Create DataFrames
    mdi_df = pd.DataFrame({'Feature': feature_names, 'Importance': mdi_importances}).sort_values(by='Importance', ascending=False)
    perm_df = pd.DataFrame({'Feature': feature_names, 'Importance': perm_importances}).sort_values(by='Importance', ascending=False)
    
    # 3. Visualization: Side-by-Side Comparison 
    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    
    # MDI Plot
    sns.barplot(ax=axes[0], x='Importance', y='Feature', data=mdi_df, palette="viridis")
    axes[0].set_title('Mean Decrease in Impurity (MDI)')
    axes[0].set_xlabel('Importance')
    
    # Permutation Plot
    sns.barplot(ax=axes[1], x='Importance', y='Feature', data=perm_df, palette="magma")
    axes[1].set_title('Permutation Importance (AUC Drop)')
    axes[1].set_xlabel('Mean AUC Drop')

    plt.suptitle("Random Forest Feature Importance Comparison (13 Ranks)")
    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout for suptitle
    
    save_path = os.path.join(BRONZE_DIR, "visualizations", "random_forest_mdi_perm_comparison.png")
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Random Forest Comparison visualization saved to {save_path}")

    return rf_model

def analyze_logreg_coefficients(X_train, y_train, BRONZE_DIR):
    """
    Trains the final Logistic Regression model (via Pipeline) and visualizes its coefficients.
    """
    logreg_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression(solver='liblinear', random_state=42))
    ])
    
    print("Training final Logistic Regression model for coefficient analysis...")
    logreg_pipeline.fit(X_train, y_train)
    
    coefficients = logreg_pipeline['logreg'].coef_[0]
    feature_names = X_train.columns
    
    # Create a DataFrame for easy sorting and plotting
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    }).sort_values(by='Coefficient', ascending=False)
    
    # Visualization 
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Coefficient', y='Feature', data=coef_df, palette="coolwarm")
    plt.title("Logistic Regression Coefficients (Impact on Playoff Odds)")
    plt.xlabel("Coefficient Value (Standardized)")
    plt.tight_layout()
    
    save_path = os.path.join(BRONZE_DIR, "visualizations", "logreg_coefficients.png")
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Logistic Regression Coefficient visualization saved to {save_path}")
    
    return logreg_pipeline # Return the pipeline

def persist_model(clf):
    """
    Saves the model using Joblib.
    """
    print(f"Saving model to {MODEL_OUTPUT}...")
    joblib.dump(clf, MODEL_OUTPUT, compress=3)
    print("Model serialized successfully.")


def plot_roc_curve(clf, X_test, y_test, model_name, BRONZE_DIR):
    """
    Generates and saves the ROC-AUC curve for the final model.
    """
    # 1. Calculate the AUC score
    y_probs = clf.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_probs)
    
    # 2. Plot the ROC Curve
    plt.figure(figsize=(8, 8))
    roc_display = RocCurveDisplay.from_estimator(
        clf, X_test, y_test, 
        name=f'{model_name} (AUC = {auc_score:.4f})'
    )
    
    # 3. Add formatting and the random guess line
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess (AUC = 0.5)')
    plt.title(f'ROC Curve for {model_name} Playoff Prediction')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.legend(loc="lower right")
    plt.grid(True)
    
    # 4. Save the plot
    save_path = os.path.join(BRONZE_DIR, "visualizations", f"{model_name.lower().replace(' ', '_')}_roc_curve.png")
    plt.savefig(save_path)
    plt.close()
    print(f"✅ {model_name} ROC Curve visualization saved to {save_path}")

if __name__ == "__main__":
    # 1. Load and Sanitize Data
    # X_pd and y_pd are the cleaned Pandas DataFrame/Series of the same length (N rows)
    X_pd, y_pd, meta = load_and_sanitize(INPUT_FILE)
    
    # 🛑 CRITICAL FIX: Convert to pure NumPy arrays for reliable slicing in TSCV.
    # We use .ravel() on y to guarantee a clean 1-dimensional vector (N,) shape.
    X = X_pd.values
    y = y_pd.values.ravel()
    
    print("-" * 50)
    print(f"Loaded Data Shapes - X: {X.shape}, y: {y.shape}")
    if X.shape[0] != y.shape[0]:
        # This check should now only fail if load_and_sanitize returned mismatched lengths
        print("CRITICAL ERROR: X and Y have inconsistent lengths!")
        raise ValueError(f"X ({X.shape[0]} samples) and Y ({y.shape[0]} samples) have inconsistent lengths after cleaning.")
    print("-" * 50)


    # Define the models to test for the TSCV comparison
    models_to_test = {
        "RandomForest": RandomForestClassifier(n_estimators=200, max_depth=12, min_samples_split=5, random_state=42, n_jobs=-1),
        "GradientBoosting": GradientBoostingClassifier(random_state=42),
        "LogisticRegression": LogisticRegression(solver='liblinear', random_state=42)
    }
    
    all_results = {}
    
    # 2. PERFORM TIME SERIES CROSS-VALIDATION (TSCV)
    print("\n" + "="*50)
    print(f"STARTING TIME SERIES CROSS-VALIDATION (k=5 folds)")
    print("="*50)
    
    for name, model in models_to_test.items():
        # X and y are the NumPy arrays
        all_results[name] = run_tscv_metrics(model, X, y)
        
    # 3. PRINT THE FINAL COMPARISON TABLE
    print_tscv_table(all_results)
    
    # 4. Split the data for final training/testing (using the NumPy arrays X and y)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # 5. RUN BOTH FEATURE IMPORTANCE ANALYSES
    print("\n" + "="*50)
    print("FINAL FEATURE IMPORTANCE COMPARISON")
    print("="*50)
    
    # CONVERSION: Convert NumPy arrays back to Pandas DataFrames for visualization functions
    # This is necessary so the plot labels (feature names) are correctly carried over.
    X_train_pd = pd.DataFrame(X_train, columns=X_pd.columns)
    X_test_pd = pd.DataFrame(X_test, columns=X_pd.columns)
    
    # Analyze with Random Forest
    _ = analyze_random_forest_comparison_plot(X_train_pd, y_train, BRONZE_DIR) 

    # Analyze with the BEST model (Logistic Regression)
    final_model_pipeline = analyze_logreg_coefficients(X_train_pd, y_train, BRONZE_DIR)
    
    # 6. Final Evaluation and Persistence (using the BEST model: LogReg)
    print("\n" + "="*50)
    print("FINAL LOGISTIC REGRESSION TEST SET EVALUATION")
    print("="*50)
    
    # Use the Pandas test set here because the final_model_pipeline (LogReg) uses a StandardScaler
    # which performs best when consistently trained/tested on DataFrames (though NumPy is often fine).
    evaluate_performance(final_model_pipeline, X_test_pd, y_test)
    
    plot_roc_curve(final_model_pipeline, X_test_pd, y_test, "Logistic Regression", BRONZE_DIR)
    
    persist_model(final_model_pipeline)

Loading data from /Users/brockolson/Desktop/STAT766/Stat 766 Final Project/nba_modeling_data_silver.csv...
Features Selected (13): ['EFG_PCT', 'FTA_RATE', 'TM_TOV_PCT_x', 'OREB_PCT', 'OPP_EFG_PCT', 'OPP_FTA_RATE', 'OPP_TOV_PCT', 'OPP_OREB_PCT', 'AST_TO', 'AST_RATIO', 'E_PACE', 'PACE_PER40', 'POSS']
Final Data Shapes: X=(450, 13), y=(450,)
--------------------------------------------------
Loaded Data Shapes - X: (450, 13), y: (450,)
--------------------------------------------------

STARTING TIME SERIES CROSS-VALIDATION (k=5 folds)
-> Evaluating RandomForestClassifier with TSCV...
-> Evaluating GradientBoostingClassifier with TSCV...
-> Evaluating LogisticRegression with TSCV...

FINAL TIME SERIES CROSS-VALIDATION RESULTS (Mean Score ± Std Dev)
| Model | Avg. Accuracy | Avg. Precision | Avg. Recall | Avg. F1 Score | Avg. ROC-AUC |
|---|---|---|---|---|---|
| RandomForest | 0.7627 (±0.0408) | 0.7480 (±0.0490) | 0.8438 (±0.0823) | 0.7892 (±0.0348) | 0.8546 (±0.0379) |
| GradientBoosting

/var/folders/bn/0fvjr47j2t15yg29h6rg2kfw0000gn/T/ipykernel_23072/427532257.py:240: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(ax=axes[0], x='Importance', y='Feature', data=mdi_df, palette="viridis")
/var/folders/bn/0fvjr47j2t15yg29h6rg2kfw0000gn/T/ipykernel_23072/427532257.py:245: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(ax=axes[1], x='Importance', y='Feature', data=perm_df, palette="magma")


✅ Random Forest Comparison visualization saved to /Users/brockolson/Desktop/STAT766/Stat 766 Final Project/visualizations/random_forest_mdi_perm_comparison.png
Training final Logistic Regression model for coefficient analysis...
✅ Logistic Regression Coefficient visualization saved to /Users/brockolson/Desktop/STAT766/Stat 766 Final Project/visualizations/logreg_coefficients.png

FINAL LOGISTIC REGRESSION TEST SET EVALUATION

MODEL EVALUATION METRICS
Accuracy:  0.8222
Precision: 0.8200 (Low False Positives)
Recall:    0.8542 (Low False Negatives)
F1 Score:  0.8367
ROC-AUC:   0.9221 (Area Under the Curve)

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.79      0.80        42
           1       0.82      0.85      0.84        48

    accuracy                           0.82        90
   macro avg       0.82      0.82      0.82        90
weighted avg       0.82      0.82      0.82        90

✅ Logistic Regression ROC Curve visua

/var/folders/bn/0fvjr47j2t15yg29h6rg2kfw0000gn/T/ipykernel_23072/427532257.py:282: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Coefficient', y='Feature', data=coef_df, palette="coolwarm")


<Figure size 800x800 with 0 Axes>